### Task 1:
Consider a standard deck of 52 cards.

What is the probability of drawing a red card **bold text**(hearts or diamonds)?

Now, given that you have drawn a red card, what is the probability that the card is a heart?

Given that the drawn card is a face card (Jack, Queen, King), what is the probability that it is a diamond?

Given that the card drawn is a face card, calculate the probability that it is also a spade or a queen.

In [ ]:
def calculate_probabilities():
    total_cards = 52
    red_cards = 26
    hearts = 13
    face_cards = 12
    diamond_face_cards = 3
    spade_face_cards = 3
    queens = 4

    p_red = red_cards / total_cards

    p_heart_given_red = hearts / red_cards

    p_diamond_given_face = diamond_face_cards / face_cards

    spade_or_queen_face = spade_face_cards + queens - 1
    p_spade_or_queen_given_face = spade_or_queen_face / face_cards

    return {
        "P(Red card)": p_red,
        "P(Heart | Red card)": p_heart_given_red,
        "P(Diamond | Face card)": p_diamond_given_face,
        "P(Spade or Queen | Face card)": p_spade_or_queen_given_face
    }

probabilities = calculate_probabilities()
for event, prob in probabilities.items():
    print(f"{event}: {prob:.3f}")

### Task 2

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = BayesianNetwork([
    ('Intelligence', 'Grade'),
    ('StudyHours',   'Grade'),
    ('Difficulty',   'Grade'),
    ('Grade',        'Pass')
])


cpd_I = TabularCPD('Intelligence', 2, [[0.7], [0.3]],
                   state_names={'Intelligence': ['High', 'Low']})

cpd_S = TabularCPD('StudyHours', 2, [[0.6], [0.4]],
                   state_names={'StudyHours': ['Sufficient', 'Insufficient']})

cpd_D = TabularCPD('Difficulty', 2, [[0.4], [0.6]],
                   state_names={'Difficulty': ['Hard', 'Easy']})

cpd_G = TabularCPD('Grade', 3,
    [
      [0.90, 0.80, 0.70, 0.60, 0.60, 0.50, 0.40, 0.30],
      [0.08, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.40],
      [0.02, 0.05, 0.10, 0.15, 0.10, 0.15, 0.20, 0.30]
    ],
    evidence=['Intelligence', 'StudyHours', 'Difficulty'],
    evidence_card=[2,2,2],
    state_names={
      'Grade': ['A','B','C'],
      'Intelligence': ['High','Low'],
      'StudyHours': ['Sufficient','Insufficient'],
      'Difficulty': ['Hard','Easy']
    }
)

cpd_P = TabularCPD('Pass', 2,
    [[0.95, 0.80, 0.50],
     [0.05, 0.20, 0.50]],
    evidence=['Grade'],
    evidence_card=[3],
    state_names={'Pass':['Yes','No'], 'Grade':['A','B','C']}
)

model.add_cpds(cpd_I, cpd_S, cpd_D, cpd_G, cpd_P)
assert model.check_model()

infer = VariableElimination(model)

q1 = infer.query(
    variables=['Pass'],
    evidence={'StudyHours':'Sufficient','Difficulty':'Hard'}
)
print("P(Pass | Sufficient, Hard):")
print(q1)

q2 = infer.query(
    variables=['Intelligence'],
    evidence={'Pass':'Yes'}
)
print("\nP(Intelligence=High | Pass=Yes):")
print(q2)


### Task 3

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = BayesianNetwork([
    ('Disease', 'Fever'),
    ('Disease', 'Cough'),
    ('Disease', 'Fatigue'),
    ('Disease', 'Chills')
])


cpd_disease = TabularCPD(
    variable='Disease', variable_card=2,
    values=[[0.30],
            [0.70]],
    state_names={'Disease': ['Flu', 'Cold']}
)

cpd_fever = TabularCPD(
    'Fever', 2,
    [[0.90, 0.50],
     [0.10, 0.50]],
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Fever': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

cpd_cough = TabularCPD(
    'Cough', 2,
    [[0.80, 0.60],
     [0.20, 0.40]],
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Cough': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

cpd_fatigue = TabularCPD(
    'Fatigue', 2,
    [[0.70, 0.30],
     [0.30, 0.70]],
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Fatigue': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

cpd_chills = TabularCPD(
    'Chills', 2,
    [[0.60, 0.40],
     [0.40, 0.60]],
    evidence=['Disease'],
    evidence_card=[2],
    state_names={'Chills': ['Yes', 'No'], 'Disease': ['Flu', 'Cold']}
)

model.add_cpds(
    cpd_disease,
    cpd_fever,
    cpd_cough,
    cpd_fatigue,
    cpd_chills
)
assert model.check_model(), "CPTs are inconsistent!"

infer = VariableElimination(model)

q1 = infer.query(
    variables=['Disease'],
    evidence={'Fever': 'Yes', 'Cough': 'Yes'}
)
print("Task 1: P(Disease | Fever=Yes, Cough=Yes)")
print(q1)


q2 = infer.query(
    variables=['Disease'],
    evidence={'Fever': 'Yes', 'Cough': 'Yes', 'Chills': 'Yes'}
)
print("\nTask 2: P(Disease | Fever=Yes, Cough=Yes, Chills=Yes)")
print(q2)


q3 = infer.query(
    variables=['Fatigue'],
    evidence={'Disease': 'Flu'}
)
print("\nTask 3: P(Fatigue=Yes | Disease=Flu)")
print(q3)


### Task 4
Define a simple Markov Model with three weather states: Sunny, Cloudy, and Rainy.

Create a transition matrix for the weather states, specifying the probabilities of transitioning from one state to another (e.g., from Sunny to Cloudy, or from Cloudy to Rainy).

Simulate the weather for the next 10 days, starting with a Sunny day, using the
Markov Model. Calculate the probability of having at least 3 rainy days over the 10-day period.

In [ ]:
import numpy as np
import random

def create_weather_model():
    transition_matrix = np.array([
        [0.6, 0.3, 0.1],
        [0.3, 0.4, 0.3],
        [0.2, 0.3, 0.5]
    ])
    return transition_matrix

def simulate_weather(days, start_state=0):
    states = ['Sunny', 'Cloudy', 'Rainy']
    transition_matrix = create_weather_model()

    current_state = start_state
    weather_sequence = [states[current_state]]
    rainy_days = 1 if current_state == 2 else 0

    for _ in range(days - 1):
        next_state_probs = transition_matrix[current_state]
        current_state = random.choices(range(3), weights=next_state_probs)[0]
        weather_sequence.append(states[current_state])
        if current_state == 2:
            rainy_days += 1

    return weather_sequence, rainy_days

def calculate_probability_at_least_3_rainy_days(n_simulations=10000):
    rainy_days_count = 0
    for _ in range(n_simulations):
        _, rainy_days = simulate_weather(10)
        if rainy_days >= 3:
            rainy_days_count += 1

    return rainy_days_count / n_simulations

weather_sequence, rainy_days = simulate_weather(10)
print("Weather sequence for 10 days:")
print(weather_sequence)
print(f"Number of rainy days: {rainy_days}")

prob = calculate_probability_at_least_3_rainy_days()
print(f"\nProbability of at least 3 rainy days in 10 days: {prob:.3f}")